In [9]:
# Importing modules
import pandas as pd
import sqlite3

In [10]:
state_to_region = {
    'Maryland': 'South', 'Delaware': 'South', 'District of Columbia': 'South', 'West Virginia': 'South', 'Virginia': 'South',
    'Kentucky': 'South', 'Tennessee': 'South', 'North Carolina': 'South', 'South Carolina': 'South', 'Georgia': 'South',
    'Florida': 'South', 'Alabama': 'South', 'Mississippi': 'South', 'Louisiana': 'South', 'Arkansas': 'South',
    'Oklahoma': 'South', 'Texas': 'South', 'Montana': 'West', 'Idaho': 'West', 'Washington': 'West',
    'Oregon': 'West', 'California': 'West', 'Nevada': 'West', 'New Mexico': 'West', 'Arizona': 'West',
    'Utah': 'West', 'Colorado': 'West', 'Wyoming': 'West', 'Alaska': 'West', 'Hawaii': 'West',
    'Pennsylvania': 'Northeast', 'New Jersey': 'Northeast', 'New York': 'Northeast', 'New Hampshire': 'Northeast',
    'Vermont': 'Northeast', 'Rhode Island': 'Northeast', 'Massachusetts': 'Northeast', 'Maine': 'Northeast',
    'Connecticut': 'Northeast', 'Ohio': 'Midwest', 'Indiana': 'Midwest', 'Illinois': 'Midwest',
    'Michigan': 'Midwest', 'Wisconsin': 'Midwest', 'Minnesota': 'Midwest', 'North Dakota': 'Midwest',
    'South Dakota': 'Midwest', 'Nebraska': 'Midwest', 'Iowa': 'Midwest', 'Missouri': 'Midwest', 'Kansas': 'Midwest'
}

In [11]:
# Cleaning the DataFrame
def clean_dataframe(dataframe):
    columnstoremove = ['ST_CASE', 'COUNTY', 'CITY', 'CITYNAME', 'DAYNAME', 
                       'HOURNAME', 'MINUTENAME', 'TWAY_ID2', 'MILEPTNAME', 
                       'NHSNAME', 'SP_JURNAME', 'RELJCT1', 'RELJCT1NAME', 
                       'RELJCT2', 'RELJCT2NAME', 'ARR_HOURNAME', 'ARR_MINNAME', 
                       'LATITUDENAME', 'LONGITUDNAME']
    
    cleaned_df = dataframe.drop(columnstoremove, axis=1)

    # Add REGION and REGIONNAME columns
    cleaned_df['REGION'] = cleaned_df['STATENAME'].map(state_to_region)
    cleaned_df['REGIONNAME'] = cleaned_df['REGION']
    
    return cleaned_df

In [12]:
# Show all columns
pd.set_option('display.max_columns', None)

# Loading and cleaning CSVs
df_2021 = clean_dataframe(pd.read_csv('../data/accident2021.csv', encoding='ISO-8859-1'))
df_2020 = clean_dataframe(pd.read_csv('../data/accident2020.csv', encoding='ISO-8859-1'))
df_2019 = clean_dataframe(pd.read_csv('../data/accident2019.csv', encoding='ISO-8859-1'))

C:\Users\johan\AppData\Local\Temp\ipykernel_24028\622424428.py:7: DtypeWarning: Columns (40,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2019 = clean_dataframe(pd.read_csv('../data/accident2019.csv', encoding='ISO-8859-1'))


In [13]:
# Concatenating the dataframes with ignore_index=True
all_years_df = pd.concat([df_2021, df_2020, df_2019])
all_years_df = all_years_df.rename(columns={'ROUTENAME': 'ROADTYPE',
                               'RUR_URBNAME': 'POPULATION',
                               'HARM_EVNAME': 'ACCIDENTTYPE',
                               'TYP_INTNAME': 'INTERSECTIONTYPE',
                               'REL_ROADNAME': 'LOCATION',
                               'LGT_CONDNAME': 'LIGHTING',
                               'WEATHERNAME': 'WEATHERTYPE',
                               'NOT_HOURNAME': 'TIME'})

In [20]:
all_years_df = df['ACCIDENTTYPE'].unique()
print(all_years_df)

['Motor Vehicle In-Transport' 'Concrete Traffic Barrier' 'Pedestrian'
 'Ditch' 'Rollover/Overturn' 'Parked Motor Vehicle' 'Mail Box' 'Culvert'
 'Fence' 'Tree (Standing Only)' 'Guardrail Face'
 'Utility Pole/Light Support' 'Traffic Sign Support' 'Ground'
 'Non-Motorist on Personal Conveyance' 'Embankment'
 'Other Object (not fixed)' 'Guardrail End' 'Wall' 'Other Fixed Object'
 'Curb' 'Boulder' 'Building' 'Railway Vehicle'
 'Bridge Rail (Includes parapet)' 'Bridge Pier or Support' 'Pedalcyclist'
 'Cable Barrier' 'Live Animal' 'Fell/Jumped from Vehicle'
 'Motor Vehicle In-Transport Strikes or is Struck by Cargo, Persons or Objects Set-in-Motion from/by Another Motor Vehicle In Transport'
 'Shrubbery' 'Working Motor Vehicle' 'Reported as Unknown'
 'Road Vehicle on Rails' 'Fire Hydrant' 'Impact Attenuator/Crash Cushion'
 'Post, Pole or Other Supports' 'Bridge Overhead Structure'
 'Other Traffic Barrier' 'Traffic Signal Support' 'Fire/Explosion'
 'Unknown Fixed Object' 'Other Non-Collision'


In [14]:
# Connecting to the SQLite database
conn = sqlite3.connect('../data/accident_database.db')
all_years_df.to_sql('accidents', conn, if_exists='replace', index=False)

108930

In [15]:
# Query to select data needed for map
query = "SELECT LATITUDE, LONGITUD, STATE, REGION, YEAR, ROADTYPE, POPULATION, ACCIDENTTYPE, INTERSECTIONTYPE, LOCATION, LIGHTING, WEATHERTYPE, TIME FROM accidents"
df = pd.read_sql_query(query, conn)

In [16]:
# Exporting to JSON
df.to_json('../data/mapdata.json', orient='records')

# Closing connection
conn.close()